# Préparation des données pour construire un système de reconnaissance automatique de la parole avec Kaldi

## Avant-propos : fondamentaux de la Reconnaissance automatique de la parole

### Principe de l'approche traditionnelle

Un système de reconnaissance automatique de la parole (ASR pour Automatic Speech Recognition en anglais) est principalement utilisé pour **transcrire un flux de parole** (convertir une séquence orale en une séquence écrite).

La tâche que doit réaliser le système est donc la suivante : trouver la suite de mot la plus probable correspondant au signal acoustique continu reçu en entrée.    

Pour pouvoir faire ça, on utilise des modèles statistiques : un **modèle acoustique** qui permet de reconnaître les sons de la langue, un **modèle de langue** qui sert à connaître les suites de mots probables dans la langue et un **dictionnaire de prononciation** qui permet de savoir comment un mot se prononce dans la langue traitée.

**Afin d'implémenter un système performant, il est fortement recommandé que ces ressources soient validées par un expert linguiste natif de la langue.**

### Modèle acoustique
Un modèle acoustique est calculé à partir d'enregistrements de parole, transcrits ou non.
Un corpus de parole qui contient un ensemble d'enregistrements de parole et leurs transcriptions permet d'entraîner un modèle acoustique de façon **supervisée**.    

Les modèles acoustiques sont des modèles statistiques qui permettent d'estimer une suite de phonèmes à partir d’un flux continu de parole prononcée, préalablement segmenté en trames acoustiques. 
Les modèles hybrides à base de réseaux de neurones et de modèles de Markov cachés (CD-DNN-HMM) sont aujourd'hui couramment utilisés pour modéliser les sons d'une langue et produire la séquence de mots la plus probable. 
Avant l'intégration de réseaux de neurones, la modélisation acoustique était calculée à partir d'un mélange de gaussiennes (GMM pour Gaussian Mixture Model en anglais).    

Pour apprendre un modèle acoustique, il faut :
 - un **corpus de parole** qui se compose d'un ensemble de fichier audio dans lesquels des locuteurs natifs parlent (1 loc/fic) et leur version écrite (les transcriptions de ce qui est dit), 
 - un **dictionnaire de prononciation** qui contient un mot suivi de sa transcription phonétique (permet de représenter à l’aide de symboles lisibles par l'ordinateur la façon dont se prononce un mot)

#### Corpus de parole

Le corpus de parole doit contenir :
 -  des enregistrements audio d’énoncés lus ou exprimés de façon spontanée par des personnes sachant parler aisément dans la langue travaillée,
 -  les versions écrites de ces enregistrements (leurs transcriptions).     

##### Note sur les transcriptions
Les transcriptions sont généralement orthographiques.     
En principe, elles n'ont pas besoin d'être alignées avec les enregistrements car des études ont montrées que le moteur de reconnaissance vocale est capable de réaliser cette action avec précision.    
Néanmoins, si cette étape peut être réalisée, le jeu de données n'en sera que plus complet et mieux exploitable.      
Les mots de la transcription doivent être lexicalement justes : par exemple, si l'utilisateur dit le mot « menu » mais prononce [məny] au lieu de [møny] (c’est-à-dire en omettant de prononcer le son « eu » [ø]), la transcription doit quand même être « menu » et non « mnu » ou « m’nu ».     
Lorsque qu'aucune règle officielle ne régit l'orthographe de la langue à transcrire, des règles devront être mises en place au préalable ou alors la transcription sera réalisée en symboles phonétiques (segmentée néanmoins en mots).

### Modèle de langue
L'étape de modélisation de la langue est réalisé à partir de **corpus de textes**, aussi variés que possible, dans la langue cible (pour le cas d'un système monolingue).    

Les modèles de langue massivement utilisés sont des modèles statistiques.     
Ils permettent de mieux reconnaître la séquence de mots la plus pertinente.      
Ils sont élaborés à partir de lois de probabilités conditionnelles qui servent à calculer les probabilités d'apparition successive des mots dans la langue.     
Le modèle de langue dépend directement de la langue à reconnaître, notamment de sa grammaire.    

Un modèle n-gramme utilise des distributions de probabilités pour prédire les mots.     
La notion de *gramme* correspond à une unité qui est ici le mot (mais ce pourrait être une lettre ou une syllabe par exemple).     
L'indice *n* correspond à la longueur de la chaîne (de mots) qui va être modélisée.     
Par exemple, un modèle 5-grammes signifie que le modèle est capable d’estimer la probabilité d’apparition d’un mot sachant la séquence de 1 à 4 mots qui le précède.    

#### Quels outils pour l'estimation d'un modèle de langue ?
Pour estimer un modèle de langue, les outils les plus utilisés dans la communauté sont :
 - [SRILM](http://www.speech.sri.com/projects/srilm/) (gratuit mais licence réservée aux étudiants et académiques)
 - [KenLM](https://github.com/kpu/kenlm)
 - [MITLM](https://code.google.com/p/mitlm/)
 
##### n-gramme ou réseau neuronal ?
Bien que les Transformer soient les architectures à l'état-de-l'art aujourd'hui, des travaux ont montré, encore récemment, que les n-grammes sont plus adataptés que les réseaux de neurones pour modéliser de petits corpus (cas des langues peu dotées). Les n-grammes sont rapides et moins energivores à estimer sur un petit corpus (moins de 100 millions de mots).

### Dictionnaire de prononciation


Le dictionnaire de prononciation réunit des mots et leurs prononciations.     
Les prononciations sont représentées par une suite de symboles phonétiques qui servent à décrire les sons de la langue.    

Le dictionnaire de prononciation joue un rôle essentiel dans l'approche traditionnelle de la reconnaissance automatique de la parole car c'est lui qui permet de faire le lien entre la forme de surface d'un mot et sa représentation phonétique.     

Le dictionnaire est utilisé par deux fois :
 - une première fois, durant l'étape d'apprentissage : il permet de construire des modèles acoustiques en associant les représentations acoustiques de chaque unité lexicale avec leurs symboles phonétiques,
 - une deuxième fois, lors de l'étape de décodage : il est conjointement utilisé avec le modèle de langue et le modèle acoustique pour transcrire un signal inconnu.

#### Quels alphabet phonétique utiliser pour encoder les prononciations ?
Différents jeux de transcriptions phonétiques peuvent être utilisés pour phonétiser les mots (cf: https://en.wikipedia.org/wiki/Comparison_of_ASCII_encodings_of_the_International_Phonetic_Alphabet#Symbols).    
Pour en citer rapidement quelques-uns que l'on retrouve dans les corpus de parole : 
 - [l'Alphabet Phonétique International](https://fr.wikipedia.org/wiki/Alphabet_phon%C3%A9tique_international) (IPA en anglais) 
 - [le SAMPA](https://fr.wikipedia.org/wiki/X-SAMPA)
 - [l'ARPAbet](https://en.wikipedia.org/wiki/ARPABET)    
    
Choisissez celui qui modélise le mieux la ou les langues cible de votre système.



#### Compléter un dictionnaire existant
Pour ajouter du vocabulaire à votre dictionnaire, on peut utiliser un phonétiseur graphème-à-phonème (G2P).    
Si vous êtes confiants de la qualité du dictionnaire de prononciation que vous possédez déjà, vous pouvez entraîner un modèle G2P à partir de ce dictionnaire, puis appliquer ce modèle à votre nouvelle liste de mots. Il en résultera un nouveau fichier avec vos mots suivis de leur transcription phonétique.     
Exemple d'outil : [Phonetisaurus](https://github.com/AdolfVonKleist/Phonetisaurus)

### Entraînement et décodage

#### Entraînement 
Pour l'entraînement du système, 80 à 90% du corpus de parole est généralement utilisé.    
Dans le cas d'un entraînement supervisé, le but est que le système apprenne successivement, à partir des exemples fournis dans ce sous-ensemble sélectionné, qu'une séquence de trames acoustiques correspond à une séquence de phones, qui elle-même représente un phonème et enfin qu'une séquence de phonèmes forme un mot.     
Dans l'approche traditionnelle, un modèle de langue est couplé en sortie du modèle acoustique pour prédire la séquence de mot la plus probablement prononcée.

**À noter :** L'apprentissage de réseaux neuronaux est très gourmand en calcul et nécessite bien souvent l'utilisation de **processeurs graphiques (GPU)**, même lorsque le volume de données en entrée du réseau est faible.

#### Décodage
Les 10 à 20% du corpus de parole restants sont généralement utilisés pour le décodage.     
Le décodage est une étape compliquée car, dans le cas de la reconnaissance de parole continue (on parle aussi de système de reconnaissance à grand vocabulaire, LVCSR pour Large Vocabulary Continuous Speech Recognition en anglais), la segmentation des phrases en phonèmes ou le nombre de phonèmes dans chaque phrase n'est pas connu.    

Le but du décodage est que le système trouve la meilleure transcription pour un enregitrement donné.     
Techniquement, le problème que le système doit résoudre est de retrouver la séquence d'états qui a produit les observations données. À partir de cette séquence d'états, il est facile trouver la suite de phonèmes la plus probable qui correspond aux paramètres observés.    
Le modèle de langue et le dictionnaire de prononciation sont exploités au moment du décodage par l'algorithme de recherche de Viterbi afin de trouver dans le graphe d'états le chemin optimal qui correspond à la séquence de phonème prononcée.

##### Évaluation du système
Le système est évalué grâce à une métrique appelée Taux d'Erreur Mots (WER pour Word Error Rate en anglais).       
Une distance est calculée entre la transcription produite par le système (appelée « hypothèse ») et la transcription originale de l'enregistrement décodé (appelée « référence »).     
Dérivé de la distance de Levenshtein, ce calcul compare un à un les mots de la référence et de l'hypothèse, sur un axe vertical.    
C’est en divisant la somme des erreurs d'édition (substitution, insertion et omission) par le nombre total de mots de la référence que l’on obtient le WER de la séquence de mots à reconnaître.     
**Plus ce taux est faible, meilleure est la reconnaissance.**

## Documentations utiles


##### Tutoriel de prise en main de Kaldi
https://kaldi-asr.org/doc/kaldi_for_dummies.html
##### Prépration des données expliquée en détail
https://kaldi-asr.org/doc/data_prep.html 
##### Autre site expliquant la préparation des données et les premiers scripts d'apprentissage, d'alignement et de décodage
https://www.eleanorchodroff.com/tutorial/kaldi/training-acoustic-models.html#prepare-directories

## Pré-requis

#### Bibliothèques qui doivent être installées sur votre machine Linux
   
**autoconf** - utilitaire utilisé pour la compilation des outils qui seront utilisés dans Kaldi    
**automake** - associé à autoconf, il permet de générer des fichiers Makefile portables    
**make** - outil qui permet d'automatiser la compilation du code source de Kaldi (à partir de la config du Makefile)
**git** - gestionnaire de versions décentralisé  utilisé pour le téléchargement et l'installation de Kaldi 
**subversion** - svn, gestionnaire de versions utilisé pour le téléchargement du dépôt ALFFA/WOLOF
**libtool** - utilitaire de création de bibliothèques statiques et dynamiques       
**wget** - permet de transférer des données à l'aide des protocoles HTTP, HTTPS et FTP    
**zlib** - utilitaire de compression de données      
**bash** - shell utilisé par Kaldi   
**grep** - utilitaire de ligne de commande permettant de rechercher dans des ensembles de données en texte brut les lignes correspondant à une expression (régulière ou non)    
**perl** - langage de programmation dynamique, utilisé dans les scripts Kaldi pour le traitement des fichiers texte  
**awk** - langage de programmation utilisé pour la recherche et le traitement de motifs dans les fichiers et les flux de données     
**atlas** - automatisation et optimisation des calculs dans le domaine de l'algèbre linéaire

```bash
sudo apt-get install -y autoconf automake make cmake git subversion\
        libtool wget zlib1g-dev libatlas-base-dev libatlas-dev\
        liblapack-dev sox libsox-fmt-all
```
*Il est possible qu'elles soient déjà installées par défaut avec votre distribution (Ubuntu conseillé)*

In [ ]:
# sudo apt-get install -y autoconf automake make cmake git subversion\
#        libtool wget zlib1g-dev libatlas-base-dev libatlas-dev\
#        liblapack-dev sox libsox-fmt-all

## Installation de Kaldi

### Télécharger les sources
Aller sur https://github.com/kaldi-asr/kaldi    
Cliquer sur le bouton vert "Code", puis :
 - soit télécharger l'archive ZIP à l'endroit désiré et la décompresser avec votre outil préféré
 - soit cloner le dépôt 
 ```bash 
 git clone https://github.com/kaldi-asr/kaldi.git kaldi --origin upstream
 ```
 
### Lancer l'installation
Ouvrir un terminal (shell Bash) puis taper les commandes suivantes :
```bash
cd kaldi
mkdir -p build && cd build
cmake -GNinja -DCMAKE_INSTALL_PREFIX=../dist ..
cmake --build . --target install
```
*Pour plus d'information sur la compilation de Kaldi, voir https://kaldi-asr.org/doc/build_setup.html*


### Contenu du répetoire principal : kaldi
**egs** - répertoire des recettes (exemples de scripts) permettant de construire rapidement des systèmes ASR    
**misc** - répertoire fourre-tout, non pertinent pour ce tuto    
**src** - code source de Kaldi    
**tools** - composants utiles et outils externes    
**windows** - outils pour exécuter Kaldi sous Windows    (non recommandé)

## Les recettes
Si vous avez installé Kaldi à la racine de votre espace personnel, les recettes se trouveront alors dans : ```/home/${user}/kaldi/egs```     
Pour voir toutes les recettes proposées dans Kaldi :
```bash 
ls ~/kaldi/egs/
```
Dans ce tutoriel, nous utiliserons la recette qui se trouve dans le répertoire ```wsj/s5```.

<img src="misc/s5_dir_tree.png" width=400 height=400>

### Liste des éléments du répertoire wsj/s5

In [ ]:
%ls ~/kaldi/egs/wsj/s5/

## Mise en place de l'environnement pour Kaldi
<!-- img src="misc/working_dir_tree.png"> -->

### Creation de l'environnement de travail
Nous allons créer l'environnement de travail suivant :   
- **kaldi_wolof_asr/ : répertoire racine**
    - **raw/** : contiendra les sources    
        - *speech/* : corpus de parole (enregistrements associés à leurs transcriptions orthographiques)
        - *language/* : corpus de textes et/ou modèle de langue si déjà fourni
        - *lexicon/* : dictionnaire de prononciation (mot suivi de sa transcription phonétique)
    - **data/** : contiendra les données qui seront utilisées en entrée de Kaldi    
        - *train/* : données d'entraînement
        - *test/* : données de test
        - *lang/* : sera créé pendant la préparation des données. Contiendra des fichiers générés à partir du dictionnaire de prononciation (modèle de prononciation, le vocabulaire, les mots hors vocabulaire, la liste des phonèmes en contexte, ...).
            - *phones/* : fichiers relatifs à la modélisation des phones   
            - *topo/* : topologie du modèle de prononciation   
        - *local/* : sera créé pendant la préparation des données
            - *lang/* : sera créé pendant la préparation des données. Contiendra des fichiers généré à partir du dictionnaire de prononciation (lexique de prononciation, liste des phonèmes, ...)
    - **local/** : contiendra d'autres types de fichiers utiles, comme des scripts de traitement
    - **exp/** : contiendra les données en sortie de Kaldi
    - **steps/** : lien symbolique vers le dossier *steps/* du répertoire ```wsj/s5```
    - **utils/** : lien symbolique vers le dossier *utils/* du répertoire ```wsj/s5```


_Note : les dossiers **steps/** et **utils/** contiennent les scripts et utilitaires qui seront utilisés pendant la préparation des données, l'apprentissage et le décodage._

In [ ]:
%%bash
kaldi_wd="kaldi_wolof_asr"  # création du répertoire racine de travail
mkdir $kaldi_wd
mkdir -p $kaldi_wd/raw/{speech,language,lexicon} 
mkdir -p $kaldi_wd/data/{train,test}
mkdir -p $kaldi_wd/exp/
mkdir -p $kaldi_wd/local/

cd $kaldi_wd
kaldi_s5=~/kaldi/egs/wsj/s5  # sous-tend que votre kaldi est installé à la racine de votre espace personnel (HOME)
ln -s $kaldi_s5/steps  # lien symbolique vers les scripts Kaldi
ln -s $kaldi_s5/utils  # lien symbolique vers les utilitaires Kaldi
ln -s $kaldi_s5/conf  # lien symbolique vers le dossier de configuration des descripteurs acoustiques
ln -s $kaldi_s5/utils/run.pl  # lien symbolique vers le script qui permet de paralléliser les jobs sur une seule et même machine
cp $kaldi_s5/path.sh .  # fichier qui permet de mettre la variable d'environnement PATH à jour avec les chemins des outils Kaldi
cp $kaldi_s5/cmd.sh .  # fichier qui permet de spécifier le type de distribution des jobs
cp $kaldi_s5/run.sh .  # fichier qui contient la succession de scripts à lancer pour apprendre et décoder un modèle de reconnaissance automatique de la parole

Concernant la parallélisation des jobs, il est possible de choisir 3 types de distributions dans le fichier ```cmd.sh``` :
 - **run.pl** : pour lancer les jobs sur une seule et même machine
 - **queue.pl** : sur moteur [Sun Grid](https://en.wikipedia.org/wiki/Oracle_Grid_Engine), pour distribuer les jobs dans la queue (utilisé lorsque la machine est partagée entre plusieurs personnes)
 - **slurm.pl** : sur [SLURM](https://en.wikipedia.org/wiki/Slurm_Workload_Manager), pour distribuer les jobs dans la queue (utilisé lorsque la machine est partagée entre plusieurs personnes)

L'apprentissage peut être coûteux en calcul. 
Kaldi divise les données en sous-paquets qu'ils distribuent dans des jobs lancés en parallèle.
1 job = 1 CPU donc veillez à ne pas choisir un nombre plus élevé de jobs que de CPU disponibles.

Il n'est pas non plus conseillé de faire plus de sous-paquets que le nombre de locuteurs dans vos ensembles de données.

Le nombre de tâches (jobs) ou de divisions dans l'ensemble de données sera spécifié plus tard lors des étapes d'apprentissage et d'alignement. 

#### Modification du fichier path.sh
Ouvrir le fichier ```path.sh```. 
 - Modifier la **ligne 1** : le chemin de la variable KALDI_ROOT afin qu'il pointe vers votre installation de Kaldi.    
Si vous avez installé Kaldi à la racine de votre espace personnel (HOME), la variable devrait être la suivante :    
```bash 
export KALDI_ROOT=/home/${user}/kaldi
```    
    
- Modifier la **ligne 3** : ajouter le chemin ```$PWD/steps:``` dans la variable PATH.    
La ligne doit correspondre à cela :     
```bash 
export PATH=$PWD/steps:$PWD/utils/:$KALDI_ROOT/tools/openfst/bin:$PWD:$PATH    
```
    
- Commenter la **ligne 6** : l'affectation de la valeur C à la locale de la machine n'est pas une bonne idée pour les langues utilisant un autre encodage que l'ASCII. Cela pourra provoquer des problèmes d'encodages lors du traitement des fichiers textuels.    
___À savoir___ : *Kaldi affecte C à la variable LC_ALL pour des raisons d'optimisation lors des tris (au moment des calculs matriciels entre autres). __Si des erreurs surviennent__ au moment de l'exécution des scripts ou si vous avez un message d'erreur indiquant un problème dans l'ordre de tri, cela peut venir de cette variable commentée. __Dans ce cas-là, décommentez et relancer l'exécution.__*    


- Ajouter une variable qui sera affectée du chemin absolu pointant vers le répertoire racine de travail, afin d'y accéder depuis n'importe quel script :
```bash 
export WORKING_DIR=`pwd`
```

- Ajouter une variable qui sera affectée du chemin absolu pointant vers le dictionnaire de prononciation, afin d'y accéder depuis n'importe quel script :
```bash 
export LEXICON_FILE=`pwd`/raw/lexicon/lexicon.txt
```

In [ ]:
!source path.sh

## Les données
### Exemple avec le corpus wolof ALFFA (African Languages in the Field: speech Fundamentals and Automation)
Source : [dépôt Github](https://github.com/getalp/ALFFA_PUBLIC/tree/master/ASR/WOLOF/data)

#### Téléchargement des données 

Pour cet atelier, nous travaillerons avec le dossier ```test``` du dépôt uniquement pour gagner du temps sur le téléchargement des données.    
Les étapes de traitement pour préparer les fichiers des autres ensembles (train et dev) seront les mêmes.

##### /!\ PENSER À DÉSACTIVER VOTRE VPN POUR CETTE ÉTAPE /!\

In [23]:
!cd kaldi_wolof_asr/raw
# Corpus de parole
!svn checkout https://github.com/getalp/ALFFA_PUBLIC/trunk/ASR/WOLOF/data/test
!mv test speech/.
# Corpus de textes et modèle de langue
!svn checkout https://github.com/getalp/ALFFA_PUBLIC/trunk/ASR/WOLOF/data/test
!mv LM/* language/. && rm -r LM
# Dictionnaire de prononciation
!wget https://raw.githubusercontent.com/getalp/ALFFA_PUBLIC/master/ASR/WOLOF/data/local/lexicon.txt
!mv lexicon.txt lexicon/.
# # Scripts utiles
# !wget https://raw.githubusercontent.com/getalp/ALFFA_PUBLIC/master/ASR/WOLOF/kaldi-scripts/{02_lexicon.sh,03_lm_preparation.sh,04_data_prep.sh}
# !mv 02_lexicon.sh lexicon/.
# !mv 03_lm_preparation.sh language/.
# !mv 04_data_prep.sh speech/.

--2022-04-22 01:23:18--  https://raw.githubusercontent.com/getalp/ALFFA_PUBLIC/master/ASR/WOLOF/kaldi-scripts/02_lexicon.sh
Resolving proxypac.si.francetelecom.fr (proxypac.si.francetelecom.fr)... 10.192.64.10
Connecting to proxypac.si.francetelecom.fr (proxypac.si.francetelecom.fr)|10.192.64.10|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 715 [text/plain]
Saving to: ‘02_lexicon.sh’

02_lexicon.sh       100%[===================>]     715  --.-KB/s    in 0s      

2022-04-22 01:23:18 (26,7 MB/s) - ‘02_lexicon.sh’ saved [715/715]

--2022-04-22 01:23:18--  https://raw.githubusercontent.com/getalp/ALFFA_PUBLIC/master/ASR/WOLOF/kaldi-scripts/03_lm_preparation.sh
Reusing existing connection to raw.githubusercontent.com:443.
Proxy request sent, awaiting response... 200 OK
Length: 805 [text/plain]
Saving to: ‘03_lm_preparation.sh’

03_lm_preparation.s 100%[===================>]     805  --.-KB/s    in 0s      

2022-04-22 01:23:18 (33,0 MB/s) - ‘03_lm_preparation

#### Contenu du jeu de données

In [9]:
!tree -d

.
├── language
├── lexicon
└── speech
    └── test
        └── wav
            ├── 05
            └── 10

7 directories


### Corpus de parole
https://github.com/getalp/ALFFA_PUBLIC/tree/master/ASR/WOLOF/data

Le corpus de parole proposé dans ALFFA est découpé en 3 sous-ensembles : 
 - un ensemble d'entraînement (**train**) d'environ 16.8h de parole prononcée par 14 locuteurs
 - un ensemble de validation (**dev**) d'environ 1.2h de parole prononcée par 2 locuteurs
 - un ensemble de test (**test**) d'environ 0.9h de parole prononcée par 2 locuteurs
 
Pour télécharger les sources :
```bash
svn checkout https://github.com/getalp/ALFFA_PUBLIC/trunk/ASR/WOLOF/data/{train,dev,test}
```

#### Exploration du contenu du dossier

In [38]:
%%bash
speech_dir=speech
echo "Contenu du répertoire '${speech_dir}' : "
ls $speech_dir
echo ""
echo "Contenu du répertoire '${speech_dir}/test' : "
ls $speech_dir/test
echo ""
for subset in $speech_dir/test; do [ -d $subset ] && echo -e "Contenu du fichier 'text' :\n"; head $subset/text; tail $subset/text; done
echo ""
echo "Il y a $(wc -l < $subset/text) transcriptions, dont $(cat $subset/text | sort | uniq -d | wc -l ) doublons dans le fichier 'text'."

Contenu du répertoire 'speech' : 
04_data_prep.sh
test

Contenu du répertoire 'speech/test' : 
README.md
test.ref
test.trs
text
wav

Contenu du fichier 'text' :

WOL_05_lect_0005  tari boo xam ne dañu lay gàgganti rekk mokkul
WOL_05_lect_0009  dafa am jamano yoo xam ni xasaw dafay ànd ak xet gu bon
WOL_05_lect_0016  bu ngeen bëggee laale demleen ca àll ba
WOL_05_lect_0021  yenn saa damay fàtte ne nekkuma senegaal
WOL_05_lect_0022  bi mu ñówee ñu daldi koy mà ndillo
WOL_05_lect_0024  bu seen pexe soti woon tey duñu fallu
WOL_05_lect_0027  boo ci sottee diw tiir gi ba noppi nga ruux ko
WOL_05_lect_0028  dafa boot xale bi santal liggéey bi keneen
WOL_05_lect_0030  nanga diisoo ak sa jëkkër bala ngay def dara
WOL_05_lect_0034  xale bi raatale na laax bi ci dënn bi
WOL_10_lect_0987  xamoon ngeen bu baax ne àttanul biib liggéey
WOL_10_lect_0988  bantu ron dëgër na moom lañuy liggéeye lal yi
WOL_10_lect_0989  dafa mel na mbaar ma gëna fooyooy lañu ko féetale
WOL_10_lect_0990  képp ku dégg woo

### Corpus de textes
https://github.com/getalp/ALFFA_PUBLIC/tree/master/ASR/WOLOF/LM

Deux corpus de textes sont partagés dans le dépôt ainsi qu'un modèle de langue.   
Le modèle de langue (fichier avec extension .arpa) est un modèle 3-grammes estimé avec SRILM à partir des deux corpus de textes sus-mentionnés.   

Les fichiers textes sont déjà nettoyés et préparés pour l'estimation d'un modèle de langue.      
Vous pouvez donc soit estimer votre propre modèle de langue avec ces corpus de textes soit utilisé le modèle de langue partagé.

Pour télécharger les sources :
```bash
svn checkout https://github.com/getalp/ALFFA_PUBLIC/trunk/ASR/WOLOF/LM
```

In [62]:
%pwd

'/home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr'

#### Exploration des corpus de textes

In [73]:
%%bash
corpus_dir=raw/language
echo ">> Contenu du répertoire '${corpus_dir}' : "
ls $corpus_dir
echo ""
echo ">> Contenu des fichiers:"
cat $corpus_dir/*txt | head
cat $corpus_dir/*txt | tail
echo ""
echo ""

echo "L'ensemble des fichiers textuels contiennent $(cat $corpus_dir/*txt | wc -w) mots sur $(cat $corpus_dir/*txt | wc -l) lignes."
echo ""
echo "Il y a $(cat $corpus_dir/*txt | sort | uniq -d | wc -l ) doublons dans les corpus textuels."

>> Contenu du répertoire 'raw/language' : 
03_lm_preparation.sh
combineV1-web-W0.9-3gram.arpa
README.md
WOL.txt
wo_SN-web_crawled.txt

>> Contenu des fichiers:
Na baral wax ji
Na bay waar wi féete ak jànj bi
Na bufta yi jib
Na buub mbalit mi
Na dem boog
Na dem suba mboog
Na des ca kër ga ba yóos bi wér
Na jél bu ko neex
Na jél tame bu sew bi
Na ku nekk indiwaale jumtukaayam
ñu raas ko fuuyu ko te ràcc def cifaay
ñu sàkku woon it ci israayil mu genn ci mbooleem suufi araab yi mu tegoon loxo atum 1967 g
ñu àngu ñoom ñépp di an calli aréen
ñëw na ba sës ci moom rikk yàlla gaw ko ci diggante ñaari xala niki loolu lépp nag màggug guddi gi lay wone kon yàlla nanu yàlla wonale ak barkeb guddi gi nu séddu ci te yàlla baaxe nu ca xam xam boobee
ótriis republik bu ótriis réew tugal óróop
Nacc afrig ciy doomam yu am yi doole ak ci kàttanam yu nit yi looloo ko doyadiloo woon ba manul woon a jàmmaarlook canc gi colonisation
Noy
Noyal ca mbicci kaw ga bul jàpp ci pet
Nàññante ak weddeente bokk nanu 

### Dictionnaire de prononciation
Le dictionnaire de prononciation est un fichier nommé *lexicon.txt* dans Kaldi.     
Il contient une liste de mots accompagnés de leurs transcriptions phonétiques.    
Le format est le suivant :
```
<word> <phone 1> <phone 2> ... <phone n>
    où :
        <word> est un mot présent au moins dans les transcriptions et au mieux dans le modèle de langue
        <phone 1> est le symbole phonétique représenant le premier phonème du mot
        <phone 2> est le symbole phonétique représenant le deuxième phonème du mot
        ... etc jusqu'à
        <phone n> est le symbole phonétique représenant le dernier phonème du mot
```

Pour télécharger les sources :
```bash
wget https://raw.githubusercontent.com/getalp/ALFFA_PUBLIC/master/ASR/WOLOF/data/local/lexicon.txt
```

#### Exploration du fichier

In [20]:
%%bash
lexicon_dir=raw/lexicon
echo "Contenu du répertoire '${lexicon_dir}' : "
ls $lexicon_dir
echo ""
echo "Contenu de 'lexicon.txt' : "
head $lexicon_dir/lexicon.txt
tail $lexicon_dir/lexicon.txt
echo ""
echo "Il y a $(wc -l < $lexicon_dir/lexicon.txt) entrées dans le dictionnaire."
echo "Il y a $(cat $lexicon_dir/lexicon.txt | sort | uniq -d | wc -l ) doublons dans le dictionnaire."

Contenu du répertoire 'lexicon' : 
lexicon.txt

Contenu de 'lexicon.txt' : 
Na	N a
Naa	N a a
NaaN	N a a N
Naam	N a a m
Naamaan	N a a m a a n
Naamaankat	N a a m a a n k a t
Naar	N a a r
Naari	N a a r i
Naarri	N a a r r i
Naata	N a a t a
ñëwul	J @ w u l
ñëwuma	J @ w u m a
ñëëw	J @ @ w
ññu	J J u
ñóddi	J o d d i
ñórndi	J o r nd i
óróop	o r o o p
óstraali	o s t r a a l i
ótiris	o t i r i s
ótrii	o t r i i

Il y a 32039 entrées dans le dictionnaire.
Il y a 0 doublons dans le dictionnaire.


## Fichiers Kaldi : text, wav.scp, utt2spk

Pour chaque ensemble (train, dev, test), 3 fichiers doivent être créé manuellement pour Kaldi : text, wav.scp, utt2spk    

Ils seront ensuite exploités par des scripts qui généreront automatiquement les fichiers restants nécessaire à l'apprentissage des modèles.

### text

Le fichier ```text``` est un fichier qui permet d'identifier une transcription à l'aide d'un identifiant unique.    
Le format est le suivant :    
```
<transcription-id> <transcription>
    où :      
        <transcription-id> représente l'identifiant de la phrase prononcée dans l'enregistrement audio 
        <transcription> correspond à la phrase prononcée dans l'enregistrement audio (la transcription)
``` 

Le dépôt ALFFA partage le fichier ```text```, donc nous n'avons pas à le formatter.    
Nous allons simplement le copier dans le dossier *data/* du répertoire de travail.    
(Nous faisons la manip pour le fichier ```text``` du dossier *test/* mais il faudrait également le faire pour *train* et *dev/*

In [82]:
%%bash
speech_dir=raw/speech
echo "Contenu du répertoire '${speech_dir}' : "
ls $speech_dir
echo ""
echo "Contenu du répertoire '${speech_dir}/test' : "
ls $speech_dir/test

# copie du fichier 'text' dans le répertoire data/test/.
for dir in "test"  # "train" "dev"
do 
    cp $speech_dir/$dir/text data/$dir/.
done

Contenu du répertoire 'raw/speech' : 
04_data_prep.sh
test

Contenu du répertoire 'raw/speech/test' : 
README.md
test.ref
test.trs
text
wav


cp: cannot stat 'raw/speech/train/text': No such file or directory
cp: cannot stat 'raw/speech/dev/text': No such file or directory


#### Contenu du fichier text

In [83]:
%pwd

'/home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr'

In [84]:
%%bash
data_dir=data
echo "Contenu du fichier 'text' :"
head $data_dir/test/text
tail $data_dir/test/text
echo ""
echo "Il y a $(wc -l < $data_dir/test/text) transcriptions, dont $(cat $data_dir/test/text | cut -d' ' -f3- | sort | uniq -d | wc -l ) doublons dans le fichier 'text'."

Contenu du fichier 'text' :
WOL_05_lect_0005  tari boo xam ne dañu lay gàgganti rekk mokkul
WOL_05_lect_0009  dafa am jamano yoo xam ni xasaw dafay ànd ak xet gu bon
WOL_05_lect_0016  bu ngeen bëggee laale demleen ca àll ba
WOL_05_lect_0021  yenn saa damay fàtte ne nekkuma senegaal
WOL_05_lect_0022  bi mu ñówee ñu daldi koy mà ndillo
WOL_05_lect_0024  bu seen pexe soti woon tey duñu fallu
WOL_05_lect_0027  boo ci sottee diw tiir gi ba noppi nga ruux ko
WOL_05_lect_0028  dafa boot xale bi santal liggéey bi keneen
WOL_05_lect_0030  nanga diisoo ak sa jëkkër bala ngay def dara
WOL_05_lect_0034  xale bi raatale na laax bi ci dënn bi
WOL_10_lect_0987  xamoon ngeen bu baax ne àttanul biib liggéey
WOL_10_lect_0988  bantu ron dëgër na moom lañuy liggéeye lal yi
WOL_10_lect_0989  dafa mel na mbaar ma gëna fooyooy lañu ko féetale
WOL_10_lect_0990  képp ku dégg woote bi na wuyusi ci ndaje mi
WOL_10_lect_0991  bi ñu agsee daldi dugg ci kër gi seet seet gisuñu kenn
WOL_10_lect_0992  foofa fépp ñaay

## Préparation des autres fichiers

Pour générer les autres fichiers, un script est disponible dans le dépôt.
Pour d'autres données, il faudra l'adapter.

Pour récupérer les sources :
```bash
wget https://raw.githubusercontent.com/getalp/ALFFA_PUBLIC/master/ASR/WOLOF/kaldi-scripts/04_data_prep.sh
```

In [49]:
# # Copie des scripts de pré-traitement dans le dossier local/ pour garder les originaux dans raw/
# !cp raw/speech/04_data_prep.sh local/.
# !cp raw/language/03_lm_preparation.sh local/.
# !cp raw/lexicon/02_lexicon.sh local/.

In [85]:
!pwd

/home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr


In [47]:
%cd ..  # /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr est désormais le répertoire courant

/home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr


### Modification du script 04_data_prep.sh

Ce script permet de générer le fichier ```utt2spk```, ```spk2utt``` et ```wav.scp``` nécessaire à Kaldi.    


#### wav.scp

Le fichier ```wav.scp``` est un fichier qui permet d'identifier le fichier audio correspondant à une transcription.    
Le format est le suivant :
```
<transcription-id> <audio-path>
    où :      
        <transcription-id> représente l'identifiant de la phrase prononcée dans l'enregistrement audio 
        <audio-path> est le chemin complet pointant vers l'enregistrement audio
```

In [90]:
!ls

cmd.sh	data  exp  local  path.sh  raw	run.pl	run.sh	steps  utils


In [93]:
%%bash
WORKING_DIR=`pwd`
wav_dir=$WORKING_DIR/raw/speech
dest_dir=$WORKING_DIR/data/test
#Built wav.scp file
echo "make wav.scp for train dev test..."
for dir in $wav_dir/test # $wav_dir/train $wav_dir/dev
do
  pushd $dir
    readlink -e wav/*/*.wav > $dest_dir/tutu1
    cat $dest_dir/tutu1 | awk -F'/' '{print $NF}' | sed 's/.wav//g' > $dest_dir/tutu2 # get the final field as the recording name, remove .wav
    paste $dest_dir/tutu2 $dest_dir/tutu1 > $dest_dir/wav.scp
    rm $dest_dir/tutu2 $dest_dir/tutu1
  popd
done
echo -e "wav.scp created for train dev test.\n"

make wav.scp for train dev test...
~/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/raw/speech/test ~/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr
~/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr
wav.scp created for train dev test.



#### utt2spk

Le fichier ```utt2spk``` est un fichier qui permet d'identifier le locuteur prononçant la phrase dans l'enregistrement.    
Le format est le suivant :
```
<transcription-id> <speaker-id>
    où :      
        <transcription-id> représente l'identifiant de la phrase prononcée dans l'enregistrement audio 
        <speaker-id> représente l'identifiant du locuteur qui prononce la phrase
```


#### spk2utt
Il permet de relier toutes les transcriptions à un locuteur.
Le format du fichier est le suivant :
```
<speaker-id> <transcription-id> <transcription-id> <transcription-id> <transcription-id> <transcription-id>
```

In [86]:
%%bash
WORKING_DIR=`pwd`
#Built text file, utt2spk file and spk2utt
echo "make utt2spk and spk2utt for train dev test..."
for dir in $WORKING_DIR/data/test
do
  cat $dir/text | cut -d' ' -f1 > $dir/utt
  cat $dir/text | cut -d'_' -f2 > $dir/spk
  paste $dir/utt $dir/spk > $dir/utt2spk
  utils/utt2spk_to_spk2utt.pl $dir/utt2spk | sort -k1 > $dir/spk2utt
  rm $dir/utt $dir/spk
done
echo -e "utt2spk and spk2utt created for train dev test.\n"

make utt2spk and spk2utt for train dev test...
utt2spk and spk2utt created for train dev test.



In [88]:
!head data/test/utt2spk

WOL_05_lect_0005	05
WOL_05_lect_0009	05
WOL_05_lect_0016	05
WOL_05_lect_0021	05
WOL_05_lect_0022	05
WOL_05_lect_0024	05
WOL_05_lect_0027	05
WOL_05_lect_0028	05
WOL_05_lect_0030	05
WOL_05_lect_0034	05


In [89]:
!head data/test/spk2utt

05 WOL_05_lect_0005 WOL_05_lect_0009 WOL_05_lect_0016 WOL_05_lect_0021 WOL_05_lect_0022 WOL_05_lect_0024 WOL_05_lect_0027 WOL_05_lect_0028 WOL_05_lect_0030 WOL_05_lect_0034 WOL_05_lect_0035 WOL_05_lect_0045 WOL_05_lect_0049 WOL_05_lect_0050 WOL_05_lect_0053 WOL_05_lect_0056 WOL_05_lect_0060 WOL_05_lect_0061 WOL_05_lect_0066 WOL_05_lect_0071 WOL_05_lect_0087 WOL_05_lect_0094 WOL_05_lect_0097 WOL_05_lect_0106 WOL_05_lect_0108 WOL_05_lect_0111 WOL_05_lect_0114 WOL_05_lect_0129 WOL_05_lect_0137 WOL_05_lect_0143 WOL_05_lect_0148 WOL_05_lect_0151 WOL_05_lect_0153 WOL_05_lect_0160 WOL_05_lect_0161 WOL_05_lect_0168 WOL_05_lect_0171 WOL_05_lect_0174 WOL_05_lect_0184 WOL_05_lect_0185 WOL_05_lect_0193 WOL_05_lect_0206 WOL_05_lect_0211 WOL_05_lect_0213 WOL_05_lect_0229 WOL_05_lect_0232 WOL_05_lect_0240 WOL_05_lect_0241 WOL_05_lect_0248 WOL_05_lect_0251 WOL_05_lect_0259 WOL_05_lect_0269 WOL_05_lect_0271 WOL_05_lect_0272 WOL_05_lect_0278 WOL_05_lect_0293 WOL_05_lect_0308 WOL_05_lect_0319 WOL_05_lect

### Modification du script 02_lexicon.sh
Ce script va générer des fichiers qui seront utiles à la création des modèles phonétiques.
 - Modifier les **lignes 9, 10, 13 et 22** :
```bash
mkdir -p $WORKING_DIR/data/local/dict
pushd $WORKING_DIR/data/local/dict
cat $LEXICON_FILE | awk '{for (i=2;i<=NF;i++) print $i}' | sort -u > nonsilence_phones.txt
cat $LEXICON_FILE | sed 's/(.)//' > lexicon.txt
```

In [57]:
!pwd

/home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr


In [58]:
%%bash
##
# Create and prepare the dict/ directory
# Once the script ran, dict/ directory will contain:
# lexicon.txt, nonsilence_phones, silence_phones.txt, optional_silence.txt
##
WORKING_DIR=`pwd`
LEXICON_FILE=$WORKING_DIR/raw/lexicon/lexicon.txt

mkdir -p $WORKING_DIR/data/local/dict

pushd $WORKING_DIR/data/local/dict

##create nonsilence_phones.txt
cat $LEXICON_FILE | awk '{for (i=2;i<=NF;i++) print $i}' | sort -u > nonsilence_phones.txt

##create silence_phones.txt
echo "SIL" > silence_phones.txt
##extra_questions.txt
touch extra_questions.txt

##lexicon.txt
rm -f lexicon.txt # if run twice, don't append to old one
cat $LEXICON_FILE | sed 's/(.)//' > lexicon.txt

##write UNK symbol
echo -e "SIL\tSIL" >> lexicon.txt
echo -e "<UNK>\tSIL" >> lexicon.txt
echo "SIL" > optional_silence.txt

popd 


~/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict ~/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr
~/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr


#### Prepare lang/ directory

La ligne suivante permet de générer des fichiers en relation avec le dictionnaire de prononciation, la relation entre les phonèmes, et permet de créer un modèle de prononciation (dans le fichier L.fst).
Plus de détails ici : https://kaldi-asr.org/doc/data_prep.html#data_prep_lang

```bash
utils/prepare_lang.sh data/local/dict "<UNK>" data/local/lang data/lang
```


In [100]:
%%bash
WORKING_DIR=`pwd`
source $WORKING_DIR/path.sh
utils/prepare_lang.sh $WORKING_DIR/data/local/dict "<UNK>" $WORKING_DIR/data/local/lang $WORKING_DIR/data/lang

utils/prepare_lang.sh /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict <UNK> /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/lang /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/lang
Checking /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict/silence_phones.txt ...
--> reading /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict/silence_phones.txt is OK

Checking /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict/optional_silence.txt ...
--> reading /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/local/dict/optional_silence.txt
--> text seem

fstaddselfloops /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/lang/phones/wdisambig_phones.int /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/lang/phones/wdisambig_words.int 
fstdeterminizestar /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/lang/G.fst /dev/null 
fstdeterminizestar 
fsttablecompose /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/lang/L_disambig.fst /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/lang/G.fst 


### Modification du script 03_lm_preparation.sh
Ce script permet de compiler le graphe pondéré à états finis (WFST) qui modélise la grammaire de la langue à partir du modèle de langue.

In [99]:
%%bash

WORKING_DIR=`pwd`
source $WORKING_DIR/path.sh
language_model=$WORKING_DIR/raw/language/combineV1-web-W0.9-3gram.arpa

#convert ARPA to FST format for Kaldi
cat $language_model | utils/find_arpa_oovs.pl $WORKING_DIR/data/lang/words.txt  > $WORKING_DIR/data/local/lang/oovs.txt
cat $language_model | \
  grep -v '<s> <s>' | \
  grep -v '</s> <s>' | \
  grep -v '</s> </s>' | \
  arpa2fst - | fstprint | \
  utils/remove_oovs.pl $WORKING_DIR/data/local/lang/oovs.txt | \
  utils/eps2disambig.pl | utils/s2eps.pl | fstcompile --isymbols=$WORKING_DIR/data/lang/words.txt \
  --osymbols=$WORKING_DIR/data/lang/words.txt  --keep_isymbols=false --keep_osymbols=false | \
  fstrmepsilon > $WORKING_DIR/data/lang/G.fst

#if prep_lang.sh returns G.fst is not ilabel sorted, run this to sort
fstarcsort --sort_type=ilabel $WORKING_DIR/data/lang/G.fst > $WORKING_DIR/data/lang/G_new.fst
mv $WORKING_DIR/data/lang/G_new.fst $WORKING_DIR/data/lang/G.fst

arpa2fst - 
LOG (arpa2fst[5.5.983~1-7547a]:Read():arpa-file-parser.cc:94) Reading \data\ section.
LOG (arpa2fst[5.5.983~1-7547a]:Read():arpa-file-parser.cc:149) Reading \1-grams: section.
LOG (arpa2fst[5.5.983~1-7547a]:Read():arpa-file-parser.cc:149) Reading \2-grams: section.
LOG (arpa2fst[5.5.983~1-7547a]:Read():arpa-file-parser.cc:149) Reading \3-grams: section.
remove_oovs.pl: removed 3511 lines.


### Extraction des paramètres MFCC
MFCC = Mel-Frequency Cepstral Coefficients     
Les coefficients cepstraux représentent les phonèmes (unités distinctes de son) comme la forme du conduit vocal (qui est responsable de la production du son).    
Ce sont les descripteurs les plus couramment extraits dans l'approche traditionnelle.    
Pour plus d'infos : https://maelfabien.github.io/machinelearning/Speech9/#6-mel-frequency-cepstral-coefficients-mfcc

In [103]:
%%bash

WORKING_DIR=`pwd`

## compute mfcc
echo "compute mfcc for train dev test..."

for dir in $WORKING_DIR/data/test  # data/dev data/train
do
  steps/make_mfcc.sh $dir mfcc_log mfcc
  echo -e "MFCCs computation for ${dir} done.\n"
  steps/compute_cmvn_stats.sh $dir mfcc_log mfcc
  echo -e "Statistics computation for cepstral mean and variance normalization for ${dir} done.\n"
  utils/fix_data_dir.sh $dir
done
echo -e "compute mfcc done.\n"

compute mfcc for train dev test...
steps/make_mfcc.sh /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/test mfcc_log mfcc
utils/validate_data_dir.sh: Successfully validated data-directory /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/test
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for test
MFCCs computation for /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/test done.

steps/compute_cmvn_stats.sh /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/test mfcc_log mfcc
Succeeded creating CMVN stats for test
Statistics computation for cepstral mean and variance normalization for /home/gwmf9814/expes/scripts/notebooks/contribuling2022/kaldi_wolof_asr/data/test done.

fix_data_dir.sh: kept all 846 utterances.
fix_data_dir.sh: old files are kept in /home/gwmf9814/expes/scripts/not

### Lancement des phases d'apprentissage et de décodage
Il ne reste plus qu'à lancer le script qui permet d'apprendre et de décoder successivement les modèles appris par Kaldi.   
Avant toute chose, il faut encore modifier ce script. 

- Modifier la **ligne** 3 du script ```run.sh``` pour qu'il commence à la phase 1 (la phase 0 correspond à la préparation des données que l'on vient d'effectuer) :
```bash
stage=1
```
- Modifier toutes les lignes pour que les algorithmes appellent les bons ensemble de de données
Par exemple, **ligne 98**, changer :
```
data/train_si84_2kshort data/lang_nosp exp/mono0a
```
par
```
data/train data/lang exp/mono0a
```
        
     
Ensuite vous pourrez lancer le script :
```bash
. ./run.sh
```


#### Alternative pour commencer
Le dépôt ALFFA propose les scripts un à un pour comprendre les étapes pas à pas.   
Voir : https://github.com/getalp/ALFFA_PUBLIC/tree/master/ASR/WOLOF/kaldi-scripts    
    
La totalité des étapes d'apprentissage et de décodage se trouve également dans un fichier *run.sh* qui est plus adapté aux données ALFFA en wolof.     
Il se trouve dans https://github.com/getalp/ALFFA_PUBLIC/tree/master/ASR/WOLOF.

**Néanmoins ces scripts ne sont pas les derniers en date proposé par Kaldi.**